# Rideshare Analysis

## Introduction
I a working as an analyst for Zuber, a new ride-sharing company that's launching in Chicago. I will historical data on taxi rides in Chicago and historical weather data to find patterns to better understand passenger preferences and the impact of external factors on rides. I studied a database, analyzed data from competitors and tested a hypothesis about the impact of weather on ride frequency. 

### Stages
1. Introduction
2. Data Preprocessing
3. Exploratory Data Analysis
4. Statistical Hypotheses Testing
5. Conclusion

## Data Overview

### Parse Data

In [1]:
# import libraries
import re
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
# parse weather data from website
URL = 'https://practicum-content.s3.us-west-1.amazonaws.com/data-analyst-eng/moved_chicago_weather_2017.html'
req = requests.get(URL)
soup = BeautifulSoup(req.text,'lxml')
table = soup.find('table',attrs={'id':'weather_records'})

In [3]:
# create dataframe from parsed table
heading_table = []
content = []
for row in table.find_all('th'):
    heading_table.append(row.text)
for row in table.find_all('tr'):
    if not row.find_all('th'):
        content.append(
            [element.text for element in row.find_all('td')]
        )
weather_record = pd.DataFrame(content,columns=heading_table)

In [4]:
# general info on weather df
weather_record.info()
weather_record.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 697 entries, 0 to 696
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Date and time  697 non-null    object
 1   Temperature    697 non-null    object
 2   Description    697 non-null    object
dtypes: object(3)
memory usage: 16.5+ KB


,Date and time,Temperature,Description
0,2017-11-01 00:00:00,276.150,broken clouds
1,2017-11-01 01:00:00,275.700,scattered clouds
2,2017-11-01 02:00:00,275.610,overcast clouds
3,2017-11-01 03:00:00,275.350,broken clouds
4,2017-11-01 04:00:00,275.240,broken clouds


In [5]:
# save weather df as a .sql file
from sqlalchemy import create_engine

engine = create_engine('sqlite:///database.db')
weather_record.to_sql('weather_record', engine, if_exists='replace')

697

Performed analysis using SQL and queries were saved as .csv files

### Load SQL Query Results

In [6]:
# Load the file data
taxi = pd.read_csv('moved_project_sql_result_01.csv')
trips = pd.read_csv('moved_project_sql_result_04.csv')
loop_ohare = pd.read_csv('moved_project_sql_result_07.csv')

In [7]:
# general info on taxi df
taxi.info()
taxi.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  64 non-null     object
 1   trips_amount  64 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


,company_name,trips_amount
0,Flash Cab,19558
35,6742 - 83735 Tasha ride inc,39
19,Chicago Medallion Leasing INC,1607
13,Nova Taxi Affiliation Llc,3175
7,Sun Taxi,7701
20,Checker Taxi,1486
21,American United,1404
25,Top Cab Affiliation,978
57,Metro Group,11
55,6057 - 24657 Richard Addo,13


In [8]:
# lower characters in string columns
taxi.company_name = taxi.company_name.str.lower()
trips.dropoff_location_name = trips.dropoff_location_name.str.lower()

# check changes
print(taxi.sample(10))
print(trips.sample(10))

                                 company_name  trips_amount
60                    2733 - 74600 benny jona             7
57                                metro group            11
38                    6743 - 78771 luhak corp            33
61              5874 - 73628 sergey cab corp.             5
62  2241 - 44667 - felman corp, manuel alonso             3
36                     3591 - 63480 chuks cab            37
25                        top cab affiliation           978
26                            gold coast taxi           428
56              5997 - 65283 aw services inc.            12
28                                5 star taxi           310
   dropoff_location_name  average_trips
81           morgan park       7.700000
39        jefferson park      74.333333
22        lincoln square     356.733333
59               chatham      23.200000
55               dunning      30.166667
24            west ridge     298.333333
7          museum campus    1510.000000
36               ken

In [9]:
# check for duplicates in all dfs
print(taxi.duplicated().sum())
print(trips.duplicated().sum())
print(loop_ohare.duplicated().sum())

0
0
197


Duplicates in the loop_ohare are fine because the start time of the rows have been rounded to the hearest hour and are still individual rides. 

In [10]:
# check for implicit duplicates in taxi & trips dfs
print(taxi.company_name.duplicated().sum())
print(trips.dropoff_location_name.duplicated().sum())

0
0


In [11]:
# change start_ts column in loop_ohare df to date time
loop_ohare.start_ts = pd.to_datetime(loop_ohare.start_ts,format='%Y-%m-%d %H:%M:%S')
# check changes
loop_ohare.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   start_ts            1068 non-null   datetime64[ns]
 1   weather_conditions  1068 non-null   object        
 2   duration_seconds    1068 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 25.2+ KB


In [12]:
# remove leading numbers from taxi company names
def remove_nums(string):
    new_string = re.sub('\d+ - \d+ ','',string)
    new_string = re.sub('- ','',new_string)
    return new_string
taxi.company_name = taxi.company_name.apply(remove_nums)
# check changes
taxi.company_name

0                           flash cab
1           taxi affiliation services
2                    medallion leasin
3                          yellow cab
4     taxi affiliation service yellow
                   ...               
59                    adwar h. nikola
60                         benny jona
61                   sergey cab corp.
62         felman corp, manuel alonso
63                     rc andrews cab
Name: company_name, Length: 64, dtype: object

## Exploratory Data Analysis

In [13]:
import plotly.express as px

In [14]:
# top 10 dropoff locations in terms of average trips
top10_loc = trips.sort_values('average_trips',ascending=False).head(10)
bar1 = px.bar(top10_loc,x='dropoff_location_name',y='average_trips',
              title='Top 10 Dropoff Locations',color='dropoff_location_name',
              labels={'dropoff_location_name':'Dropoff Location',
                      'average_trips':'Average Trips'})
bar1.update_layout(title_font_size=20,bargap=0.1,height=600)
bar1.update_xaxes(tickangle=45)
bar1.show()

The top locations in terms of dropoffs are the Loop, River North, Streeterville & West Loop neighborhoods. When looking at a Chicago neighborhood map this makes sense. These 4 neighborhoods are all in the heart of the city in what would be considered within the downtown area. The next most popular dropoff location is the O'Hare neighborhood. This is the only top 10 neighborhood that isn't close to Lake Michigan. This still makes sense to be a popular dropoff location as this is where Chicago's International Airport is located. The remaining of the top 10 neighborhoods are in close vicinity to the downtown area and are all very close to the coast of Lake Michigan. Many of these locations are popular tourist areas, so also make sense to be popular destinations.

In [15]:
# bar graph of taxi companies and number of rides
bar2 = px.bar(taxi,x='company_name',y='trips_amount',title='Number of Trips Per Company',
              color='company_name',
              labels={'trips_amount':'Number of Trips','company_name':'Company Name'})
bar2.update_layout(title_font_size=20,bargap=0.1,
                   height=700)
bar2.update_xaxes(tickangle=45)
bar2.show()

# bar graph of the top 15 taxi companies by number of rides
bar3 = px.bar(taxi.sort_values('trips_amount',ascending=False).head(15),x='company_name',
              y='trips_amount',title='Top 15 Taxi Companies by Rides',
              color='company_name',labels={'company_name':'Company Name',
                                           'trips_amount':'Number of Trips'})
bar3.update_layout(title_font_size=20,bargap=0.15,height=700)
bar3.update_xaxes(tickangle=45)
bar3.show()

The top taxi company, Flash Cab, has almost twice the number of rides as the 2nd taxi company, Taxi Affiliation Services. Since they have such such a large proportion of the rides as compared to any other taxi company, I am guessing that they have a larger fleet of vehicles than any other taxi company. While there are lots of different taxi companies for the area (64 total), more than half of the companies do not even do 25% of the rides that the top taxi company does. 

## Hypothesis Testing

<p>Test Hypothesis: The average duration of rides from the Loop to O'Hare International Airport changes on rainy Saturdays</p>

- Null Hypothesis: The average duration of rides from the Loop to O'Hare International Airport on rainy Saturdays & not rainy Saturdays are equal
- Alternative Hypothesis: The average duration of rides from the Loop to O'Hare International Airport on rainy Saturdays & not rainy Saturdays are not equal
- Alpha (Significance) Value: 5 percent

In [16]:
from scipy import stats as st

In [17]:
# create filtered dfs from loop_ohare df of rides with good weather
good_weather = loop_ohare[loop_ohare.weather_conditions=='Good']['duration_seconds']
bad_weather = loop_ohare[loop_ohare.weather_conditions=='Bad']['duration_seconds']

In [18]:
# boxplots of loop_ohare df by good vs bad weather conditions
box1 = px.box(loop_ohare,y='weather_conditions',x='duration_seconds',
              title='Ride Durations by Weather Condition',
              color='weather_conditions',labels={'weather_conditions':'Weather Conditions',
                                                 'duration_seconds':'Trip Duration (in seconds)'})
box1.update_layout(title_font_size=20)
box1.show()

In [19]:
# test the hypotheses
alpha = 0.05
results1 = st.ttest_ind(good_weather,bad_weather)
print(f'p-value: {results1.pvalue}')

p-value: 6.517970327099473e-12


The pvalue is lower than the determined alpha value. We can reject the null hypothesis and therefore can accept the alternative hypothesis and can determine that the average duration of rides is different on rainy Saturdays.

## Conclusion
<p> Woking with the results of several SQL queries, I was able to find the number of taxi rides for each taxi company for November 15-16, 2017. Using this SQL result I was able to find the top taxi companies in terms of rides for the Chicago area. I was also able to obtain ride data for rides from the Loop to O'Hare neighborhoods, including the trip duration and weather conditions. 
</p>
<p> Using the results from the SQL queries, I am able to determine that it would be best to concentrate Zuber taxis in the downtown area of Chicago, on the east side of the city and on the coast of Michigan Lake. All of the top dropoff locations, except the O'Hare Airport, are located in these areas. It would be smart to also have a dedicated number of cars concentrated at the airport as well. I am also able to determine that the biggest competitor will be the company, Flash Cab. This taxi company is currently the leader in number of rides in the Chicago area with the second taxi company (Taxi Affiliation Services) having almost half the amount of rides. It would be smart to analyze the rides for Flash Cab to see if their taxis are concentrated in a specific area or compare the number of taxis that they have with the second highest competitor, Taxi Affiliation Services. 
</p>